In [12]:
#functions needed 
# computing words within 2 edit distance of a word 
# given a sentence compute the probability of the sentence using bi-gram modelling

In [13]:
# non-word errors 

# (1) Find the error word by comparing each word with the dictionary 
# (2) Compute the possible replacement of the word by the edit distance metric 
# (3) Rank the candidates using probability of edit distance 
# (3) For the top k words, find the sentence probability. 
# (4) Select the best sentence and proceed. 

In [14]:
import nltk

In [15]:
nltk.download('gutenberg')
nltk.download('wordnet')
nltk.download('wordnet_ic')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
files = nltk.corpus.gutenberg.fileids()
print(files)

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [17]:
from nltk.corpus import gutenberg

In [18]:
emma_words = gutenberg.words('austen-emma.txt')
len(emma_words)

192427

In [19]:
# (1) Preprocess all the sentences to create vocabulary 

# Stemming 
# lemmatization 
# remove numbers 
# lowercase 
# remove punctuation 
# remove stopwords

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import re

stopwords = set(nltk.corpus.stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def preprocess_sentence(text):

  text = text.lower()
 
  text = re.sub(r'[^\w\s]', '', text)

  text = re.sub(r'[0-9]', '', text)

  #remove unwanted spaces
  text = re.sub(' +', ' ', text)

  #remove trailing spaces 
  text = text.strip()

  #remove stop words
  f = [w for w in text.split(" ") if w not in stopwords]

  text = " ".join(f)

  # text = text.split(" ")
  # text = [stemmer.stem(word) for word in text]
  # text = [lemmatizer.lemmatize(word) for word in text]
  return text

In [20]:
emma = nltk.corpus.gutenberg.sents('austen-emma.txt')
print(emma)


[['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']'], ['VOLUME', 'I'], ...]


In [21]:
#create vocab
vocab = {}

for sentence in emma:

  res = preprocess_sentence(" ".join(sentence))

  for word in res.split(" "):
    if word in vocab:
      vocab[word] += 1
    else:
      vocab[word] = 1

In [22]:
#return all words in the vocabulary that lie within edit distance 2 of the given word

def edit_distance1(word):

    l   = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in l]
    inserts    = [L + c + R               for L, R in splits for c in l]
    return set(deletes + transposes + replaces + inserts)

def edit_distance2(word): 
    return (e2 for e1 in edit_distance1(word) for e2 in edit_distance1(e1))

def candidate_words(word, vocab):

  a1 = list(edit_distance1(word))
  a2 = list(edit_distance2(word))

  s1 = set(vocab.keys())
  s2 = set(a1 + a2)

  return list(s1.intersection(s2)) 

In [23]:

def editDistDP(str1, str2, m, n):
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]
 
    for i in range(m + 1):
        for j in range(n + 1):

            if i == 0:
                dp[i][j] = j  
 
            elif j == 0:
                dp[i][j] = i   

            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]

            else:
                dp[i][j] = 1 + min(dp[i][j-1],        
                                   dp[i-1][j],        
                                   dp[i-1][j-1])    
 
    return dp[m][n]
 
 


In [24]:
# function to return top k candidate words

def top_k(incorrect_word,vocab,k):

  def sort_f(correct_word):

    val = editDistDP(incorrect_word, correct_word, len(incorrect_word), len(correct_word))
    return val

  cws = candidate_words(incorrect_word, vocab)
  cws.sort(key = sort_f)

  return cws if k > len(cws) else cws[:k]

In [25]:
top_k("acress", vocab, 3)

['across', 'agrees', 'aches']

In [26]:
#prepare train and test corpus 
emma = nltk.corpus.gutenberg.sents('austen-emma.txt')

from sklearn.model_selection import train_test_split

train, test = train_test_split(emma, test_size=0.2, random_state=23)


In [27]:
train_processed = [preprocess_sentence(" ".join(s)) for s in train]
train_processed[2]

'_you_ probably less surprized us suspicions forgotten tried give caution wish attended sinking voice heavy sigh seem doomed blindness'

In [28]:
# train bigram model and calculate sentence probability 

import itertools

class Bigram:

  def __init__(self, train_corpus):
    self.tc = train_corpus
    self.prob = self.train_bigram()

  def pairwise(self,iterable):
      "s -> (s0, s1), (s1, s2), (s2, s3), ..."
      a, b = itertools.tee(iterable)
      next(b, None)
      return zip(a, b)  

  def train_bigram(self):

    #unigram dict
    unigrams = {}
    prob = {}
    for sentence in self.tc:

      for word in sentence.split(" "):
        if word in unigrams:
          unigrams[word] += 1
        else:
          unigrams[word] = 1
      
    for word in unigrams:
      prob[word] = unigrams[word] / sum(unigrams.values())

    #bigrams dict 

    bigrams = {}
    for sentence in train_processed:

      for pair in self.pairwise(sentence.split(" ")):

        if pair in bigrams:
          bigrams[pair] += 1
        else:
          bigrams[pair] = 1

    for pair in bigrams:
      prob[pair] = bigrams[pair] / unigrams[pair[0]]

    return prob

  def sentence_prob(self, sentence):

    #prob is the probability dictionary which is the result of training

    sentence = preprocess_sentence(sentence)
    p = 1
    split_s = sentence.split(" ")

    if split_s[0] not in self.prob:
      p *= pow(10, -4)
    else:
      p *= self.prob[split_s[0]]

    if split_s[-1] not in self.prob:
      p *= pow(10, -4)
    else:
      p *= self.prob[split_s[-1]]
    
    for pair in self.pairwise(sentence.split(" ")):

      if pair not in self.prob:
        p *= pow(10,-6)
      else:
        p *= self.prob[pair]

    return p 

In [29]:
train_processed[1]

'doubt whether sense would corrected without'

In [30]:
bigram = Bigram(train_processed)

In [31]:
def correct_sentence(sentence,vocab):

  split_s = sentence.split(" ")

  for word in split_s:
    if word not in vocab:
      #this is the incorrect word
      cws = top_k(word, vocab, 5)
      ic = word
      break
  
  candidate_sentences = [sentence.replace(word, cw) for cw in cws]

  candidate_sentences.sort(reverse = True, key = bigram.sentence_prob)

  return candidate_sentences


In [32]:
correct_sentence("i doubt wether my own sense would have corrected me without it", vocab)

[' doubt wether my own sense would have corrected me wthout t',
 'e doubt wether my own sense would have corrected me wethout et',
 'k doubt wether my own sense would have corrected me wkthout kt',
 'c doubt wether my own sense would have corrected me wcthout ct',
 'ix doubt wether my own sense would have corrected me wixthout ixt']

In [33]:
test_processed = [preprocess_sentence(" ".join(s)) for s in test]
test_processed[2]

'understood delayed till colonel campbell return'

In [34]:
correct_sentence("understood delayed till colonel campbell retun", vocab)

['understood delayed till colonel campbell return',
 'understood delayed till colonel campbell begun',
 'understood delayed till colonel campbell reign',
 'understood delayed till colonel campbell recur',
 'understood delayed till colonel campbell retain']

##ACCURACY Non-Word Errors

In [40]:
import random 

def accuracy_nonword(test_corpus, vocab):

  correct = 0

  tp = [preprocess_sentence(" ".join(s)) for s in test_corpus]
  
  for sentence in tp:
    #randomly choose a word and modify within 2 edit distance such that it does not lie in the vocabulary 

    split_s = sentence.split(" ")
    random_index = random.randint(0, len(split_s)-1)

    word = split_s[random_index]

    possible_words = set(list(edit_distance1(word)) + list(edit_distance2(word)))
    final_pw = list(possible_words.difference(set(vocab.keys())))

    incorrect_s = sentence.replace(word, final_pw[0])

    output_model = correct_sentence(incorrect_s, vocab)

    if not output_model:
      continue

    print("Original sentence: ", sentence)
    print("Mistyped non-word error sentence: ", incorrect_s)
    print("Model Output: ", output_model[0])
    print("\n")

    if output_model[0] == sentence:
      correct +=1

  return correct / len(tp)

In [41]:
#testing accuracy on a subset of test set

print("The accuracy is:", accuracy_nonword(test,vocab) * 100)

Streaming output truncated to the last 5000 lines.
Mistyped non-word error sentence:  surprized jane vfamirfax accepts civilities consents
Model Output:  surprized jane fairfax accepts civilities consents


Original sentence:  oh
Mistyped non-word error sentence:  vol
Model Output:  told


Original sentence:  extent admiration may take surprize day
Mistyped non-word error sentence:  txvent admiration may take surprize day
Model Output:  extent admiration may take surprize day


Original sentence:  hope retract said
Mistyped non-word error sentence:  hope retract saee
Model Output:  hope retract see


Original sentence:  owned considering every thing absolutely without inclination party
Mistyped non-word error sentence:  owned considering every thing absolutely witsoiut inclination party
Model Output:  owned considering every thing absolutely without inclination party


Original sentence:  gave understand frank admired extremely thought beautiful charming much said altogether found must

## Real World Error

In [42]:
# (1) Change the word to something present in the vocab 
# (2) for each word generate candidate words within 1,2 edit distance and original word.... 3 in each
# (3) generate all possible combinations and find the best sentence according to probability 

def accuracy_realword(test_corpus, vocab):

  correct = 0
  tp = [preprocess_sentence(" ".join(s)) for s in test_corpus]

  for sentence in tp:
    #randomly choose a word and modify within 2 edit distance such that it lies in the vocabulary. 

    split_s = sentence.split(" ")
    random_index = random.randint(0, len(split_s)-1)

    word = split_s[random_index]

    possible_words = set(list(edit_distance1(word)) + list(edit_distance2(word)))
    final_pw = list(possible_words.intersection(set(vocab.keys())))

    incorrect_s = sentence.replace(word, final_pw[0])

    #for each word in the incorrect sentence generate all possible sentences and find the highest probability 

    #candidate word 1 
    #candidate word 2

    inc_splits = incorrect_s.split(" ")
    sentences_probable = [] 

    for w in inc_splits:
      for cw in top_k(w,vocab,2):
        sentences_probable.append(incorrect_s.replace(w,cw))

    sentences_probable.sort(reverse = True, key = bigram.sentence_prob)

    print("Original sentence: ", sentence)
    print("Mistyped non-word error sentence: ", incorrect_s)
    print("Model Output: ", sentences_probable[0])
    print("\n")

    if sentences_probable[0] == sentence:
      correct +=1
    
  return correct / len(tp)


In [43]:
print("Accuracy for Real World is:", accuracy_realword(test,vocab))

Streaming output truncated to the last 5000 lines.
Mistyped non-word error sentence:  extent adoration may take surprize day
Model Output:  extent adoration may taken surprize day


Original sentence:  hope retract said
Mistyped non-word error sentence:  hows retract said
Model Output:  shows retract said


Original sentence:  owned considering every thing absolutely without inclination party
Mistyped non-word error sentence:  owes considering every thing absolutely without inclination party
Model Output:  owes considering every thing absolutely without inclination party


Original sentence:  gave understand frank admired extremely thought beautiful charming much said altogether found must judge harshly
Mistyped non-word error sentence:  gave understand frank admired extremely thought beautiful charming much said altogether found must judge harshly
Model Output:  gave understand frank admired extremely thought beautiful charming much said altogether found must judge harshly


Original 